In [7]:
import friktionless

In [8]:
import pandas as pd

In [9]:
dates = pd.read_gbq(
    query='select distinct date(timestamp) as date from solana.deposits',
    project_id='friktion-dev'
    )

In [10]:
dates.describe()

,date
count,110
unique,110
top,2022-04-04
freq,1


In [11]:
users = pd.read_gbq(
    query='select distinct(userAddress) as userAddress from solana.deposits')

In [12]:
users.describe()

,userAddress
count,13893
unique,13893
top,wGm3yCrVppHYratwKAv9pBNRbge18rpn9XiAujZ6jwF
freq,1


In [13]:
user_first_deposits_dates = pd.read_gbq(
    query='select userAddress, min(date(timestamp)) as first_deposit_dt from solana.deposits group by userAddress')

In [14]:
user_first_deposits_dates.describe()

,userAddress,first_deposit_dt
count,13893,13893
unique,13893,110
top,wGm3yCrVppHYratwKAv9pBNRbge18rpn9XiAujZ6jwF,2022-03-30
freq,1,2158


In [15]:
users.iloc[0][0]

'wGm3yCrVppHYratwKAv9pBNRbge18rpn9XiAujZ6jwF'

In [16]:
user_first_deposits_dates[user_first_deposits_dates['userAddress'].str.contains(users.iloc[0][0])]

,userAddress,first_deposit_dt
0,wGm3yCrVppHYratwKAv9pBNRbge18rpn9XiAujZ6jwF,2022-04-04


In [17]:
# eventuall you'll iterate through this activity of making a user table per userAddress and then unioning together with all the others

In [18]:
# to make this iterate through all users increment the first value in the iloc call
current_user_address = users.iloc[0][0]
user_first_deposit_date = user_first_deposits_dates[user_first_deposits_dates['userAddress'].str.contains(current_user_address)]['first_deposit_dt']

In [19]:
user_deposit_table = pd.DataFrame(columns=['userAddress', 'asOfDate', 'totalValueLockedUSD', 'totalDepositedUSD', 'totalWithdrawnUSD', \
                                           'tvlDelta30Days', 'tvlDelta60Days', 'tvlDelta90Days', \
                                           'tvlDelta1Epoch', 'tvlDelta2Epoch', \
                                           'tvlDelta3Epoch', 'tvlDelta4Epoch', 'tvlDelta5Epoch', \
                                           'firstDepositDate', 'firstDepositEpoch', 'firstDepositAmount', 'firstDepositToken', \
                                           'lastDepositDate', 'lastDepositEpoch', 'lastDepositAmount', 'lastDepositToken', \
                                           'daysSinceLastDeposit', 'epochsSinceLastDeposit', \
                                           'firstWithdrawalDate', 'firstWithdrawalEpoch', 'firstWithdrawalAmount', 'firstWithdrawalToken', \
                                           'lastWithdrawalDate', 'lastWithdrawalEpoch', 'lastWithdrawalAmount', 'lastWithdrawalToken', \
                                           'hasChurned', 'churnDate', 'churnEpoch'])

### asOfDate

In [20]:
user_deposit_table['asOfDate'] = dates[dates['date'].apply(lambda x: x>=user_first_deposit_date).iloc[:,0]]

### userAddress

In [21]:
user_deposit_table['userAddress'] = current_user_address

### totalValueDepositedUSD
TODO: Add logic to covert to USD

#### inner loop logic

In [22]:
query_date = user_deposit_table['asOfDate'][0].isoformat()
pd.read_gbq(query=f"select sum(amount) as total_deposited from solana.deposits where userAddress=\'{current_user_address}\' and date(timestamp) <= date(\'{query_date}\')")

,total_deposited
0,1.0


In [23]:
# need to add market marking code
user_deposit_table['totalDepositedUSD'] = [pd.read_gbq(query=f"select sum(amount) as total_deposited from solana.deposits where userAddress=\'{current_user_address}\' and date(timestamp) <= date(\'{query_date.isoformat()}\')").iloc[0,0] for query_date in user_deposit_table['asOfDate'].tolist()]

### totalValueWithdrawnUSD
TODO: Add logic to convert to USD

In [24]:
query_date = user_deposit_table['asOfDate'][0].isoformat()
pd.read_gbq(query=f"select sum(amount) as total_withdrawn from solana.withdrawals where userAddress=\'{current_user_address}\' and date(timestamp) <= date(\'{query_date}\')").fillna(0).iloc[0,0]

0.0

In [25]:
# need to add market marking code
user_deposit_table['totalWithdrawnUSD'] = [pd.read_gbq(query=f"select sum(amount) as total_withdrawn from solana.withdrawals where userAddress=\'{current_user_address}\' and date(timestamp) <= date(\'{query_date.isoformat()}\')").fillna(0).iloc[0,0] for query_date in user_deposit_table['asOfDate'].tolist()]

### totalValueLockedUSD

In [26]:
user_deposit_table['totalValueLockedUSD'] = user_deposit_table['totalDepositedUSD'] - user_deposit_table['totalWithdrawnUSD']

### firstDepositDate

In [36]:
first_deposit_date = user_first_deposits_dates[user_first_deposits_dates['userAddress'].str.match(current_user_address)]['first_deposit_dt'].iloc[0]
user_deposit_table['firstDepositDate'] =  first_deposit_date

### firstDepositEpoch

### firstDepositAmount

In [41]:
user_deposit_table['firstDepositAmount'] = pd.read_gbq(query=f'select sum(amount) from solana.deposits where userAddress = \'{current_user_address}\' and date(timestamp) = \'{first_deposit_date.isoformat()}\'').iloc[0,0]

### firstDepositToken

In [44]:
user_deposit_table['firstDepositToken'] = pd.read_gbq(query=f'select currencyName from solana.deposits where userAddress = \'{current_user_address}\' and date(timestamp) = \'{first_deposit_date.isoformat()}\'').iloc[0,0]

### tvlDelta30Days

### tvlDelta60Days

### tvlDelta90Days

In [45]:
user_deposit_table.melt()

,variable,value
0,userAddress,wGm3yCrVppHYratwKAv9pBNRbge18rpn9XiAujZ6jwF
1,asOfDate,2022-04-04
2,totalValueLockedUSD,1.0
3,totalDepositedUSD,1.0
4,totalWithdrawnUSD,0.0
5,tvlDelta30Days,NaN
6,tvlDelta60Days,NaN
7,tvlDelta90Days,NaN
8,tvlDelta1Epoch,NaN
9,tvlDelta2Epoch,NaN


### tvlDelta1Epoch

### tvlDelta2Epoch

### tvlDelta3Epoch

### tvlDelta4Epoch

### tvlDelta5Epoch

### lastDepositDate

### lastDepositEpoch

### lastDepositAmount

### lastDepositToken

### daysSinceLastDeposit

### epochsSinceLastDeposit

### firstWithdrawalDate

### firstWithdrawalEpoch

### firstWithdrawalAmount

### firstWithdrawalToken

### lastWithdrawalDate

### lastWithdrawalEpoch

### lastWithdrawalAmount

### lastWithdrawalToken

### hasChurned

### chrunDate

### churnEpoch